<a href="https://colab.research.google.com/github/idrisdemir/KolektifProje/blob/main/Kollektifproje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00


In [ ]:
#libraries
import torch
from torch import nn
from torch.nn.parameter import Parameter
import numpy as np
from torch.utils.data import DataLoader,StackDataset
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
import time
import random
import matplotlib.pyplot as plt

#from mpl_toolkits.mplot3d import Axes3D


In [ ]:
batch_size = 16
epochs = 10
loss_fn = nn.CrossEntropyLoss()
num_ara_agirlik = 5


In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# ***Veri Seti***

In [ ]:
dataset = load_dataset("ag_news",split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

# ***Prepare Veri Seti***

In [ ]:
#metin kümesinin vektör temsilleri elde ediliyor
vectorizer = CountVectorizer(max_features=1000,dtype='float32')
vectorizer.fit(dataset['text'])
text_vectors = vectorizer.transform(dataset['text']).toarray()

In [ ]:
#Veri setinin %15 'i test için ayrıldı

train_x, train_y = text_vectors[:45000],dataset['label'][:45000] #120000'in yüzde 15 i 102000'dir.
test_x, test_y = text_vectors[45000:50000],dataset['label'][45000:50000]

In [ ]:
train_dataset = StackDataset(text=train_x,label=train_y)
test_dataset = StackDataset(text=test_x,label=test_y)

In [ ]:
train_dataset = DataLoader(train_dataset,batch_size=batch_size)
test_dataset = DataLoader(test_dataset,batch_size=batch_size)

# ***Model***

In [ ]:
# Define model
class TextNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(1000,256) #her bir vectör kısmında farklı değerler oluyor
        self.linear_2 = nn.Linear(256,256)
        self.linear_3 = nn.Linear(256,256)
        self.linear_4 = nn.Linear(256,4)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.relu(x)
        x = self.linear_3(x)
        x = self.relu(x)
        logits = self.linear_4(x)
        return logits
    def get_parameters(self):
      linear1_w = np.reshape(self.linear_1.weight.cpu().detach().numpy(),(-1,))
      linear1_b = np.reshape(self.linear_1.bias.cpu().detach().numpy(),(-1,))
      linear2_w = np.reshape(self.linear_2.weight.cpu().detach().numpy(),(-1,))
      linear2_b = np.reshape(self.linear_2.bias.cpu().detach().numpy(),(-1,))
      #linear3_w = np.reshape(self.linear_3.weight.cpu().detach().numpy(),(-1,))
      #linear3_b = np.reshape(self.linear_3.bias.cpu().detach().numpy(),(-1,))

      weights = np.concatenate((linear1_w,linear1_b,linear2_w,linear2_b))#,linear3_w,linear3_b))
      return weights

    def load_parameters(self,parameters):
      self.linear_1.weigth = Parameter(torch.tensor(parameters[:1000*32]))
      self.linear_1.bias = Parameter(torch.tensor(parameters[1000*32:1000*32 + 32]))
      self.linear_2.weight = Parameter(torch.tensor(parameters[1000*32 + 32:1000*32 + 32 + 32*4]))
      self.linear_2.bias = Parameter(torch.tensor(parameters[1000*32 + 32 + 32*4:]))

# ***Training***

In [ ]:
#veri kümesi ve model veriliyor. O model o veri kümesi ile eğitiliyor. Eğitilen model döndürülüyor.
def train(ds,model):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  for epoch in range(epochs):
    start_time = time.perf_counter()
    print(f"Epoch {epoch+1}\n-------------------------------")
    size = len(ds.dataset)
    model.train()
    for batch_no, batch in enumerate(ds):
      X, y = batch['text'].to(device), batch['label'].to(device)

      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch_no % 100 == 0:
          loss, current = loss.item(), (batch_no + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
  return model

In [ ]:
model1 = TextNetwork().to(device)
model1 = train(train_dataset,model1)

Epoch 1
-------------------------------
loss: 1.406728  [   16/45000]
loss: 1.383899  [ 1616/45000]
loss: 1.272114  [ 3216/45000]
loss: 0.964173  [ 4816/45000]
loss: 0.458655  [ 6416/45000]
loss: 0.424463  [ 8016/45000]
loss: 0.611285  [ 9616/45000]
loss: 0.521508  [11216/45000]
loss: 0.375147  [12816/45000]
loss: 0.661358  [14416/45000]
loss: 0.915623  [16016/45000]
loss: 0.304290  [17616/45000]
loss: 0.105810  [19216/45000]
loss: 0.732774  [20816/45000]
loss: 0.261520  [22416/45000]
loss: 0.360640  [24016/45000]
loss: 0.182595  [25616/45000]
loss: 0.178499  [27216/45000]
loss: 0.409011  [28816/45000]
loss: 0.372151  [30416/45000]
loss: 0.188707  [32016/45000]
loss: 0.153324  [33616/45000]
loss: 0.491376  [35216/45000]
loss: 0.755903  [36816/45000]
loss: 0.410531  [38416/45000]
loss: 0.136615  [40016/45000]
loss: 0.168791  [41616/45000]
loss: 0.333948  [43216/45000]
loss: 0.458618  [44816/45000]
Epoch 2
-------------------------------
loss: 0.435932  [   16/45000]
loss: 0.636636  [ 16

In [ ]:
model2 = TextNetwork().to(device)
model2 = train(train_dataset,model2)

Epoch 1
-------------------------------
loss: 1.436506  [   16/45000]
loss: 1.388877  [ 1616/45000]
loss: 1.277072  [ 3216/45000]
loss: 1.012687  [ 4816/45000]
loss: 0.446284  [ 6416/45000]
loss: 0.392911  [ 8016/45000]
loss: 0.617245  [ 9616/45000]
loss: 0.562449  [11216/45000]
loss: 0.383127  [12816/45000]
loss: 0.632760  [14416/45000]
loss: 0.954364  [16016/45000]
loss: 0.353934  [17616/45000]
loss: 0.094388  [19216/45000]
loss: 0.748617  [20816/45000]
loss: 0.242672  [22416/45000]
loss: 0.390976  [24016/45000]
loss: 0.157113  [25616/45000]
loss: 0.177881  [27216/45000]
loss: 0.446204  [28816/45000]
loss: 0.391772  [30416/45000]
loss: 0.182957  [32016/45000]
loss: 0.156912  [33616/45000]
loss: 0.467106  [35216/45000]
loss: 0.734961  [36816/45000]
loss: 0.408248  [38416/45000]
loss: 0.125796  [40016/45000]
loss: 0.155604  [41616/45000]
loss: 0.357471  [43216/45000]
loss: 0.461564  [44816/45000]
Epoch 2
-------------------------------
loss: 0.437972  [   16/45000]
loss: 0.654383  [ 16

In [ ]:
model3 = TextNetwork().to(device)
model3 = train(train_dataset,model3)

Epoch 1
-------------------------------
loss: 1.432007  [   16/45000]
loss: 1.378582  [ 1616/45000]
loss: 1.240448  [ 3216/45000]
loss: 1.043665  [ 4816/45000]
loss: 0.486539  [ 6416/45000]
loss: 0.462969  [ 8016/45000]
loss: 0.687769  [ 9616/45000]
loss: 0.493755  [11216/45000]
loss: 0.374116  [12816/45000]
loss: 0.634559  [14416/45000]
loss: 0.939290  [16016/45000]
loss: 0.308079  [17616/45000]
loss: 0.104184  [19216/45000]
loss: 0.703902  [20816/45000]
loss: 0.209180  [22416/45000]
loss: 0.366999  [24016/45000]
loss: 0.193739  [25616/45000]
loss: 0.167458  [27216/45000]
loss: 0.378231  [28816/45000]
loss: 0.397073  [30416/45000]
loss: 0.177970  [32016/45000]
loss: 0.164025  [33616/45000]
loss: 0.470193  [35216/45000]
loss: 0.723596  [36816/45000]
loss: 0.401524  [38416/45000]
loss: 0.120388  [40016/45000]
loss: 0.155828  [41616/45000]
loss: 0.318112  [43216/45000]
loss: 0.458572  [44816/45000]
Epoch 2
-------------------------------
loss: 0.445812  [   16/45000]
loss: 0.624516  [ 16

In [ ]:
model4 = TextNetwork().to(device)
model4 = train(train_dataset,model4)

Epoch 1
-------------------------------
loss: 1.422003  [   16/45000]
loss: 1.377235  [ 1616/45000]
loss: 1.267353  [ 3216/45000]
loss: 1.026864  [ 4816/45000]
loss: 0.429069  [ 6416/45000]
loss: 0.402245  [ 8016/45000]
loss: 0.666140  [ 9616/45000]
loss: 0.528172  [11216/45000]
loss: 0.351584  [12816/45000]
loss: 0.695682  [14416/45000]
loss: 0.895312  [16016/45000]
loss: 0.334760  [17616/45000]
loss: 0.102209  [19216/45000]
loss: 0.701583  [20816/45000]
loss: 0.233439  [22416/45000]
loss: 0.375943  [24016/45000]
loss: 0.141013  [25616/45000]
loss: 0.169679  [27216/45000]
loss: 0.411697  [28816/45000]
loss: 0.365742  [30416/45000]
loss: 0.182647  [32016/45000]
loss: 0.149218  [33616/45000]
loss: 0.475765  [35216/45000]
loss: 0.719365  [36816/45000]
loss: 0.404251  [38416/45000]
loss: 0.126525  [40016/45000]
loss: 0.167869  [41616/45000]
loss: 0.331931  [43216/45000]
loss: 0.461758  [44816/45000]
Epoch 2
-------------------------------
loss: 0.494445  [   16/45000]
loss: 0.644995  [ 16

In [ ]:
model5 = TextNetwork().to(device)
model5 = train(train_dataset,model5)

Epoch 1
-------------------------------
loss: 1.356480  [   16/45000]
loss: 1.364033  [ 1616/45000]
loss: 1.303742  [ 3216/45000]
loss: 0.994202  [ 4816/45000]
loss: 0.435677  [ 6416/45000]
loss: 0.415779  [ 8016/45000]
loss: 0.601479  [ 9616/45000]
loss: 0.578302  [11216/45000]
loss: 0.384545  [12816/45000]
loss: 0.681242  [14416/45000]
loss: 0.916897  [16016/45000]
loss: 0.299846  [17616/45000]
loss: 0.095346  [19216/45000]
loss: 0.718825  [20816/45000]
loss: 0.235588  [22416/45000]
loss: 0.382102  [24016/45000]
loss: 0.156249  [25616/45000]
loss: 0.174633  [27216/45000]
loss: 0.419575  [28816/45000]
loss: 0.384692  [30416/45000]
loss: 0.193715  [32016/45000]
loss: 0.151573  [33616/45000]
loss: 0.478307  [35216/45000]
loss: 0.730009  [36816/45000]
loss: 0.399898  [38416/45000]
loss: 0.121062  [40016/45000]
loss: 0.156097  [41616/45000]
loss: 0.335828  [43216/45000]
loss: 0.474661  [44816/45000]
Epoch 2
-------------------------------
loss: 0.463931  [   16/45000]
loss: 0.658562  [ 16

# ***Test***

In [ ]:
def test_score(ds,model):
  size = 0
  test_loss, accuracy = 0,0
  with torch.no_grad():
    for sample in ds:

      X = sample['text']
      y = sample['label']
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      accuracy += (pred.argmax(-1) == y).type(torch.float).sum().item() #batch_accuracy
      size += len(X)
  accuracy = accuracy / size
  return accuracy, size

In [ ]:
accuracy, size = test_score(test_dataset,model1)
accuracy

0.8324

In [ ]:
accuracy, size = test_score(test_dataset,model2)
accuracy

0.838

In [ ]:
accuracy, size = test_score(test_dataset,model3)
accuracy

0.84

In [ ]:
accuracy, size = test_score(test_dataset,model4)
accuracy

0.8238

In [ ]:
accuracy, size = test_score(test_dataset,model5)
accuracy

0.8338

In [ ]:
PATH = "drive/MyDrive/Kollektif3modeller/n5"
torch.save(model5,PATH)